# AULA 3 - Prática 3: Introdução à Otimização para Redes Neurais

Aula 3 - Aula Assíncrona
https://www.youtube.com/watch?v=O3h4kUdSV4k&t=2s&ab_channel=MoacirAntonelliPonti

---

## Otimização para Redes Neurais

- Redes Densas em pytorch e detalhes:
    - Camadas pré-implementadas
    - Opções para acessar parâmetros
    - Uso do gradiente
    - Backpropagation e algoritmos de otimização
    
---

In [2]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

### Projeto Rede Densa

- Exemplo da aula teórica
    - Entrada: imagens 28x28 pixels
    - Saída: 10 classes (dígitos de 0 a 9)

In [3]:
class Network(nn.Module): # herda uma classe com diversos parâmetros encapsulados

    def __init__(self):
        # inicialização conforme a superclasse nn.Module
        super(Network, self).__init__()
        # cria o projeto da rede neural
        # duas camadas Fully Connected (camadas densas), fc1 = hidden e fc2 = final
        self.fc1 = nn.Linear(784, 32) # entrada = 784, saída = 32
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        # achatar todas as camadas, exceto uma
        x = torch.flatten(x, 1) # recebe minibatches
        x = F.relu(self.fc1(x)) # camada densa (processa os dados de x) + relu
        x = self.fc2(x) # camada densa linear (sem softmax por enquanto)
        return x

In [4]:
net = Network()
net

Network(
  (fc1): Linear(in_features=784, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=10, bias=True)
)

In [7]:
input_random = torch.randn(1, 1, 28, 28) # 1 batch com 1 canal de cor 28x28 = imagem aleatoria
print(input_random.shape)

output = net(input_random)
output

torch.Size([1, 1, 28, 28])


tensor([[-0.2961, -0.1091,  0.5809, -0.0283, -0.1638,  0.0520,  0.1414, -0.1080,
         -0.0909, -0.0965]], grad_fn=<AddmmBackward0>)

- Acessando os parâmetros da rede

In [8]:
params = list(net.parameters())
print(len(params))
# primeira camada
print(params[0].size())
print(params[0])

4
torch.Size([32, 784])
Parameter containing:
tensor([[ 0.0243, -0.0092,  0.0023,  ..., -0.0019,  0.0040, -0.0288],
        [ 0.0141, -0.0345,  0.0016,  ..., -0.0143,  0.0107, -0.0199],
        [-0.0244, -0.0241, -0.0177,  ..., -0.0020,  0.0112, -0.0288],
        ...,
        [ 0.0291, -0.0108,  0.0162,  ...,  0.0101,  0.0351, -0.0249],
        [-0.0114,  0.0357, -0.0221,  ..., -0.0325, -0.0244,  0.0228],
        [-0.0318,  0.0357,  0.0213,  ..., -0.0258, -0.0139,  0.0129]],
       requires_grad=True)


### Backpropagation

* Já está pronto: `backward`

In [9]:
# zerar o buffer de gradientes
net.zero_grad()
output.backward(torch.randn(1,10)) # aleatorio para exemplo

### Rede operando

In [12]:
output = net(input_random)
target = torch.randn(10) # target aleatorio como exemplo
print(target)
target = target.view(1, -1) # um elemento em uma dimensão e o resto em outra
print(target)

criterion = nn.MSELoss() # mean squared error
loss = criterion(output, target)
print(loss)

tensor([ 1.2478, -0.3516, -0.9182, -1.0940,  0.5680, -0.2292, -0.9411, -0.1779,
         0.1158,  1.4357])
tensor([[ 1.2478, -0.3516, -0.9182, -1.0940,  0.5680, -0.2292, -0.9411, -0.1779,
          0.1158,  1.4357]])
tensor(1.0007, grad_fn=<MseLossBackward0>)


In [13]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0]) # func linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # func relu

### Backpropagation com `backwards`

In [14]:
net.zero_grad() # tem que zerar o gradiente 
# antes
print('Antes:', net.fc1.bias.grad)

loss.backward()
print('Depois:', net.fc1.bias.grad)

Antes: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])
Depois: tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0628,  0.0000,  0.0760,  0.0000,
         0.0000,  0.0083, -0.0303,  0.0396,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0981,  0.0000, -0.0217,  0.0000, -0.0223,  0.0306,  0.0000,
         0.0862,  0.0355,  0.0780,  0.0370,  0.0715,  0.0000,  0.0000,  0.0000])


### Otimizadores

- Adaptacao dos parâmetros da rede
    - Otimziador: SGD, Adam, entre outros

In [18]:
print(params[0][0][:10])

tensor([ 0.0243, -0.0092,  0.0023, -0.0185,  0.0247,  0.0155,  0.0332,  0.0174,
        -0.0116, -0.0058], grad_fn=<SliceBackward0>)


In [19]:
import torch.optim as optim

# cria objeto otimizador
optimizer = optim.SGD(net.parameters(), lr=0.01)

# para cada loop de treinamento

# zerar o buffer dos grads
optimizer.zero_grad()
# gerar a saida e computar gradientes com relacao a func de perda
output = net(input_random)
loss = criterion(output, target)
loss.backward()
# realiza a adaptacao dos pesos
optimizer.step()

In [20]:
print(params[0][0][:10])

tensor([ 0.0243, -0.0092,  0.0023, -0.0185,  0.0247,  0.0155,  0.0332,  0.0174,
        -0.0116, -0.0058], grad_fn=<SliceBackward0>)
